In [1]:
import os
import glob
import pandas as pd
import re

In [2]:
import glob

# Ruta a la carpeta que contiene los CSV's
RawData_rstate = '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM'
OldData_rstate = '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/OldData'

# Obtener todos los archivos .csv en el directorio RawData
csv_files = glob.glob(os.path.join(RawData_rstate, '*.csv'))
csv_olds = glob.glob(os.path.join(OldData_rstate, '*.csv'))

# Lista para almacenar rutas completas de archivo
file_paths = []

# Iterar sobre los archivos y obtener las rutas completas
for file in csv_files:
    file_paths.append(file)

for file in csv_olds:
    file_paths.append(file)

# Imprimir la lista de rutas completas de archivo
print(file_paths)

['/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_alquiler_sevilla_20240709.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_alquiler_cadiz_20240626.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_alquiler_malaga_20240626.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_venta_granada_20240705.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_venta_madrid_20240708.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TF

In [3]:
# Diccionario para almacenar los dataframes con sus nombres de archivo
dataframes = {}

# Iterar sobre cada archivo CSV y crear un dataframe
for file in csv_files:
    # Extraer el nombre del archivo sin la ruta y la extensión
    file_name = file.split('/')[-1].replace('.csv', '')
    try:
        # Leer el archivo CSV en un dataframe de pandas
        df = pd.read_csv(file, delimiter=';', on_bad_lines='skip')
        
        # Determinar valor para columna 'fuente_datos'
        if 'trovit' in file_name:
            df['fuente_datos'] = 'trovit'
        else:
            df['fuente_datos'] = 'pisos.com'
        
        # Determinar valor para columna 'alquiler_venta'
        if 'alquiler' in file_name:
            df['alquiler_venta'] = 'alquiler'
        else:
            df['alquiler_venta'] = 'venta'
        
        # Determinar valor para columna 'fecha_extract'
        # Extraer la parte de la fecha del nombre del archivo
        fecha_extract_str = file_name.split('_')[-1]  # Obtener '20240630', '20240630', etc.
        # Convertir a formato de fecha
        fecha_extract = pd.to_datetime(fecha_extract_str, format='%Y%m%d')
        df['fecha_extract'] = fecha_extract
        
        # Almacenar el dataframe en el diccionario
        dataframes[file_name] = df
        
    except pd.errors.ParserError as e:
        print(f"Error parsing {file}: {e}")

In [4]:
for file in csv_olds:
    # Extraer el nombre del archivo sin la ruta y la extensión
    file_name = file.split('/')[-1].replace('.csv', '')
    try:
        # Leer el archivo CSV en un dataframe de pandas
        df = pd.read_csv(file, delimiter=';', on_bad_lines='skip')
        
        df['fuente_datos'] = 'pisos.com'
        
        # Determinar valor para columna 'alquiler_venta'
        if 'alquiler' in file_name:
            df['alquiler_venta'] = 'alquiler'
        else:
            df['alquiler_venta'] = 'venta'
        
        # Determinar valor para columna 'fecha_extract'
        # Extraer la parte de la fecha del nombre del archivo
        fecha_extract_str = file_name.split('_')[-1]  # Obtener '20240630', '20240630', etc.
        # Convertir a formato de fecha
        fecha_extract = pd.to_datetime(fecha_extract_str, format='%Y%m%d')
        df['fecha_extract'] = fecha_extract
        
        # Almacenar el dataframe en el diccionario
        dataframes[file_name] = df
        
    except pd.errors.ParserError as e:
        print(f"Error parsing {file}: {e}")

In [5]:
# Convertir el diccionario de dataframes en una lista de dataframes
dfs = list(dataframes.values())

# Concatenar todos los dataframes en uno solo
consolidated_df = pd.concat(dfs, ignore_index=True)

# Mostrar el dataframe consolidado
print(consolidated_df.tail())

             precio                          sub_descr  \
434730    620.000 €        Piso en calle de Cochabamba   
434731  1.070.000 €  Piso en calle del Padre Damián, 5   
434732    225.000 €              Piso en Pinar del Rey   
434733    375.000 €              Piso en Pinar del Rey   
434734    170.000 €              Piso en Pinar del Rey   

                                                     href  \
434730  /comprar/piso-chamartin_hispanoamerica28016-37...   
434731  /comprar/piso-chamartin_hispanoamerica28036-38...   
434732  /comprar/piso-pinar_del_rey28033-39205965533_1...   
434733  /comprar/piso-pinar_del_rey28033-40080751969_1...   
434734  /comprar/piso-pinar_del_rey28033-39245702964_1...   

                                                ubicacion habitaciones  \
434730  Hispanoamérica (Distrito Chamartín. Madrid Cap...      2 habs.   
434731  Hispanoamérica (Distrito Chamartín. Madrid Cap...      4 habs.   
434732  Pinar del Rey (Distrito Hortaleza. Madrid Capi...     

def extraer_numeros(columna):
    numeros_extraidos = columna.str.extract(r'(\d+\.?\d*)').astype(float)
    return numeros_extraidos


def extraer_numeros_decimales(columna):
    numeros_extraidos = columna.str.replace(',', '.')
    numeros_extraidos = numeros_extraidos.astype('float')
    return numeros_extraidos

def limpia_datos(dat):
    # Elimina duplicados
    dat.drop_duplicates(inplace = True)
    
    # Latitud y longitud a numérico
    dat['latitud_numerico'] = extraer_numeros_decimales(dat['latitude'])
    dat['longitud_numerico'] = extraer_numeros_decimales(dat['longitude'])
    
    # Extraemos los números de las columnas 'habitaciones', 'banios' y 'mt2'
    dat['habitaciones_numerico'] = extraer_numeros(dat['habitaciones'])
    dat['banios_numerico'] = extraer_numeros(dat['banios'])
    dat['mt2_numerico'] = extraer_numeros(dat['mt2'])
    
    dat['precio_numerico'] = dat['precio'].str.replace('.', '').str.extract(r'(\d+)').astype(float)
    # Chequear si todos los precios están en euros
    
    
    dat['frase_paréntesis'] = dat['ubicacion'].str.extract(r'\((.*?)\)')
    
    # Separamos la frase extraída utilizando el punto como separador
    dat[['Distrito', 'Otra_ubicacion']] = dat['frase_paréntesis'].str.split('.', n=1, expand=True)
    
    # Eliminamos los espacios en blanco alrededor de las cadenas resultantes
    dat['Distrito'] = dat['Distrito'].str.strip()
    dat['Otra_ubicacion'] = dat['Otra_ubicacion'].str.strip()
    
    # Eliminamos la columna temporal 'frase_paréntesis'
    dat.drop(columns=['frase_paréntesis'], inplace=True)
    
    # Generamos una nueva columna 'precio_por_mt2' que representa el precio por metro cuadrado
    dat['precio_por_mt2'] = dat['precio_numerico'] / dat['mt2_numerico']
    
    
    ## Guarda datos limpios y elimina columnas repetidas
    cols_to_drop = ['precio', 'habitaciones', 'banios',
           'mt2', 'planta', 'latitude', 'longitude']
    dat.drop(columns=cols_to_drop, inplace = True)
    dat.columns = dat.columns.str.replace('_numerico','')
    return dat


columns_to_clean_consolidated = ['precio', 'habitaciones', 'banios', 'mt2', 'publicado_hace', 'planta' ]
dat = limpia_datos(consolidated_df[column])

columns_to_clean_consolidated = ['precio', 'habitaciones', 'banios', 'mt2', 'publicado_hace', 'planta' ] # Asegúrate de ajustar según tus columnas reales
for column in columns_to_clean_consolidated:
    consolidated_df[column] = consolidated_df[column].apply(limpia_datos)

In [6]:
def limpiar_numeros(numero_en_sucio):
    # Manejar valores NaN y convertir a cadena si es necesario
    if pd.isna(numero_en_sucio):
        return None
    # Convertir a cadena
    numero_en_sucio = str(numero_en_sucio)
    # Eliminar cualquier texto que no sea un número, punto, coma o espacio
    numero_en_sucio = re.sub(r'[^\d.,€\s]', '', numero_en_sucio)
    # Eliminar el símbolo de euro y espacios en blanco
    numero_en_sucio = re.sub(r'[€\s]', '', numero_en_sucio)
    # Reemplazar los puntos (separadores de miles) por nada
    numero_en_sucio = numero_en_sucio.replace('.', '')
    # Reemplazar las comas (separadores decimales) por nada
    numero_en_sucio = numero_en_sucio.replace(',', '')
    # Convertir a entero, manejar excepciones si la conversión falla
    try:
        return int(numero_en_sucio)
    except ValueError:
        return None

# Aplicar la misma transformación al dataframe consolidado
columns_to_clean_consolidated = ['precio', 'habitaciones', 'banios', 'mt2', 'publicado_hace', 'planta' ] # Asegúrate de ajustar según tus columnas reales
for column in columns_to_clean_consolidated:
    consolidated_df[column] = consolidated_df[column].apply(limpiar_numeros)

    
# Convertir las columnas 'latitude' y 'longitude' a tipo float
consolidated_df['latitude'] = pd.to_numeric(consolidated_df['latitude'], errors='coerce')
consolidated_df['longitude'] = pd.to_numeric(consolidated_df['longitude'], errors='coerce')

In [7]:
consolidated_df.describe()

,precio,habitaciones,banios,mt2,latitude,longitude,planta,fecha_extract,publicado_hace
count,4.306220e+05,420527.000000,402833.000000,411606.000000,352448.000000,352448.000000,177498.000000,434735,72753.000000
mean,3.964720e+05,2.914331,2.035136,182.846581,39.941103,-3.226311,3.122289,2024-04-16 19:28:49.409410816,48.022666
min,4.500000e+01,1.000000,1.000000,1.000000,0.000000,-16.749433,1.000000,2023-06-10 00:00:00,1.000000
25%,2.545000e+03,2.000000,1.000000,73.000000,40.308757,-3.712423,1.000000,2024-02-09 00:00:00,10.000000
50%,1.790000e+05,3.000000,2.000000,101.000000,40.423981,-3.684591,3.000000,2024-06-22 00:00:00,21.000000
75%,4.150000e+05,4.000000,2.000000,160.000000,40.458305,-3.611847,4.000000,2024-07-04 00:00:00,30.000000
max,2.450000e+07,140.000000,332.000000,1000000.000000,43.508612,2.762230,20.000000,2024-07-14 00:00:00,2350.000000
std,7.544569e+05,1.524133,1.429491,5099.412813,1.682689,1.867209,2.209998,NaN,202.098720


In [8]:
consolidated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434735 entries, 0 to 434734
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   precio          430622 non-null  float64       
 1   sub_descr       390305 non-null  object        
 2   href            434735 non-null  object        
 3   ubicacion       434680 non-null  object        
 4   habitaciones    420527 non-null  float64       
 5   banios          402833 non-null  float64       
 6   mt2             411606 non-null  float64       
 7   otros           433696 non-null  object        
 8   latitude        352448 non-null  float64       
 9   longitude       352448 non-null  float64       
 10  planta          177498 non-null  float64       
 11  CCAA            236657 non-null  object        
 12  fuente_datos    434735 non-null  object        
 13  alquiler_venta  434735 non-null  object        
 14  fecha_extract   434735 non-null  dat

In [9]:
consolidated_df

,precio,sub_descr,href,ubicacion,habitaciones,banios,mt2,otros,latitude,longitude,planta,CCAA,fuente_datos,alquiler_venta,fecha_extract,publicado_hace,plataforma,raw_json
0,1200.0,NaN,/alquilar/piso-zona_esperanza_de_triana41010-4...,Zona Esperanza de Triana (Distrito Triana. Sev...,3.0,1.0,69.0,¡Alquiler para estudiantes! Contrato mínimo de...,37.379340,-6.003559,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
1,900.0,Piso en calle Ares,/alquilar/piso-bellavista_la_palmera_bellavist...,Bellavista (Distrito Bellavista-La Palmera. Se...,3.0,2.0,105.0,Magnífico piso en urbanización exclusiva: jard...,37.332449,-5.965829,5.0,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
2,1000.0,NaN,/alquilar/chalet-oromana-45028203361_517213/,Oromana (Alcalá de Guadaíra),4.0,2.0,146.0,Fantástico chalet en alquiler de larga tempora...,37.320301,-5.856279,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
3,990.0,NaN,/alquilar/piso-zona_ronda_de_triana_cartuja410...,Zona Ronda de Triana-Cartuja (Distrito Triana....,2.0,1.0,80.0,¡Descubre tu nuevo hogar en el corazón de tria...,37.389190,-6.008260,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
4,1000.0,NaN,/alquilar/piso-nervion_ciudad_jardin41005-4503...,Ciudad Jardín (Distrito Nervión. Sevilla Capital),3.0,1.0,60.0,Piso en alquiler de 70 m² en ciudad jardin sol...,37.380612,-5.961544,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434730,620000.0,Piso en calle de Cochabamba,/comprar/piso-chamartin_hispanoamerica28016-37...,Hispanoamérica (Distrito Chamartín. Madrid Cap...,2.0,2.0,108.0,Vivienda de lujo reformada exterior con traste...,NaN,NaN,2.0,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN
434731,1070000.0,"Piso en calle del Padre Damián, 5",/comprar/piso-chamartin_hispanoamerica28036-38...,Hispanoamérica (Distrito Chamartín. Madrid Cap...,4.0,4.0,200.0,Walter Haus tiene el placer de ofrecerle este ...,NaN,NaN,3.0,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN
434732,225000.0,Piso en Pinar del Rey,/comprar/piso-pinar_del_rey28033-39205965533_1...,Pinar del Rey (Distrito Hortaleza. Madrid Capi...,3.0,1.0,86.0,Agencia inmobiliaria de madrid zona parque san...,NaN,NaN,2.0,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN
434733,375000.0,Piso en Pinar del Rey,/comprar/piso-pinar_del_rey28033-40080751969_1...,Pinar del Rey (Distrito Hortaleza. Madrid Capi...,3.0,2.0,113.0,Agencia inmobiliaria de madrid zona parque san...,NaN,NaN,4.0,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN


In [10]:
consolidated_df

,precio,sub_descr,href,ubicacion,habitaciones,banios,mt2,otros,latitude,longitude,planta,CCAA,fuente_datos,alquiler_venta,fecha_extract,publicado_hace,plataforma,raw_json
0,1200.0,NaN,/alquilar/piso-zona_esperanza_de_triana41010-4...,Zona Esperanza de Triana (Distrito Triana. Sev...,3.0,1.0,69.0,¡Alquiler para estudiantes! Contrato mínimo de...,37.379340,-6.003559,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
1,900.0,Piso en calle Ares,/alquilar/piso-bellavista_la_palmera_bellavist...,Bellavista (Distrito Bellavista-La Palmera. Se...,3.0,2.0,105.0,Magnífico piso en urbanización exclusiva: jard...,37.332449,-5.965829,5.0,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
2,1000.0,NaN,/alquilar/chalet-oromana-45028203361_517213/,Oromana (Alcalá de Guadaíra),4.0,2.0,146.0,Fantástico chalet en alquiler de larga tempora...,37.320301,-5.856279,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
3,990.0,NaN,/alquilar/piso-zona_ronda_de_triana_cartuja410...,Zona Ronda de Triana-Cartuja (Distrito Triana....,2.0,1.0,80.0,¡Descubre tu nuevo hogar en el corazón de tria...,37.389190,-6.008260,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
4,1000.0,NaN,/alquilar/piso-nervion_ciudad_jardin41005-4503...,Ciudad Jardín (Distrito Nervión. Sevilla Capital),3.0,1.0,60.0,Piso en alquiler de 70 m² en ciudad jardin sol...,37.380612,-5.961544,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434730,620000.0,Piso en calle de Cochabamba,/comprar/piso-chamartin_hispanoamerica28016-37...,Hispanoamérica (Distrito Chamartín. Madrid Cap...,2.0,2.0,108.0,Vivienda de lujo reformada exterior con traste...,NaN,NaN,2.0,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN
434731,1070000.0,"Piso en calle del Padre Damián, 5",/comprar/piso-chamartin_hispanoamerica28036-38...,Hispanoamérica (Distrito Chamartín. Madrid Cap...,4.0,4.0,200.0,Walter Haus tiene el placer de ofrecerle este ...,NaN,NaN,3.0,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN
434732,225000.0,Piso en Pinar del Rey,/comprar/piso-pinar_del_rey28033-39205965533_1...,Pinar del Rey (Distrito Hortaleza. Madrid Capi...,3.0,1.0,86.0,Agencia inmobiliaria de madrid zona parque san...,NaN,NaN,2.0,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN
434733,375000.0,Piso en Pinar del Rey,/comprar/piso-pinar_del_rey28033-40080751969_1...,Pinar del Rey (Distrito Hortaleza. Madrid Capi...,3.0,2.0,113.0,Agencia inmobiliaria de madrid zona parque san...,NaN,NaN,4.0,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN


In [11]:
dropped_df = consolidated_df.drop_duplicates(subset=consolidated_df.columns.difference(['fecha_extract']))

In [12]:
dropped_df

,precio,sub_descr,href,ubicacion,habitaciones,banios,mt2,otros,latitude,longitude,planta,CCAA,fuente_datos,alquiler_venta,fecha_extract,publicado_hace,plataforma,raw_json
0,1200.0,NaN,/alquilar/piso-zona_esperanza_de_triana41010-4...,Zona Esperanza de Triana (Distrito Triana. Sev...,3.0,1.0,69.0,¡Alquiler para estudiantes! Contrato mínimo de...,37.379340,-6.003559,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
1,900.0,Piso en calle Ares,/alquilar/piso-bellavista_la_palmera_bellavist...,Bellavista (Distrito Bellavista-La Palmera. Se...,3.0,2.0,105.0,Magnífico piso en urbanización exclusiva: jard...,37.332449,-5.965829,5.0,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
2,1000.0,NaN,/alquilar/chalet-oromana-45028203361_517213/,Oromana (Alcalá de Guadaíra),4.0,2.0,146.0,Fantástico chalet en alquiler de larga tempora...,37.320301,-5.856279,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
3,990.0,NaN,/alquilar/piso-zona_ronda_de_triana_cartuja410...,Zona Ronda de Triana-Cartuja (Distrito Triana....,2.0,1.0,80.0,¡Descubre tu nuevo hogar en el corazón de tria...,37.389190,-6.008260,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
4,1000.0,NaN,/alquilar/piso-nervion_ciudad_jardin41005-4503...,Ciudad Jardín (Distrito Nervión. Sevilla Capital),3.0,1.0,60.0,Piso en alquiler de 70 m² en ciudad jardin sol...,37.380612,-5.961544,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434721,490000.0,Piso en calle de Clara del Rey,/comprar/piso-chamartin_prosperidad28002-40007...,Prosperidad (Distrito Chamartín. Madrid Capital),2.0,2.0,104.0,Br houses real estate vende piso completamente...,40.447420,-3.667489,1.0,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN
434726,2700000.0,Piso en Chamberi,/comprar/piso-chamberi_almagro28010-3834787423...,Almagro (Distrito Chamberí. Madrid Capital),4.0,4.0,280.0,Engel & Völkers tiene el placer de presentarle...,NaN,NaN,NaN,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN
434727,790000.0,Piso en Retiro,/comprar/piso-nino_jesus28007-39248582279_101800/,Niño Jesús (Distrito Retiro. Madrid Capital),5.0,3.0,170.0,Engel & Völkers presenta este gran piso para r...,NaN,NaN,NaN,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN
434728,550000.0,Piso en Centro,/comprar/piso-centro_embajadores_lavapies28012...,Embajadores-Lavapiés (Distrito Centro. Madrid ...,2.0,2.0,82.0,Engel & Völkers comercializa esta vivienda con...,NaN,NaN,NaN,NaN,pisos.com,venta,2024-01-08,NaN,NaN,NaN


In [13]:
dropped_df.to_csv('consolidated_data.csv', index=False)